In [2]:
#from libs.prefect_helpers import *
from libs.data_manipulation import *
from libs.feature_generation import *
from libs.dim_reduction import *
from libs.football_plots import *
from libs.clustering import *
from libs.convex_hull import *

import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
from libs.InteractivePitchUI import InteractivePitch, PitchDisplay

from mplsoccer import *
import pandas as pd
import numpy as np
import os
# Click player into the pitch and take those player and apply Ripleys K to them. And then maybe PCA


In [ ]:
# Step 6: Extract match data from tracking data
data = compile_team_tracking_data("data/H_EURO2024GERMANY", "Denmark")
print(data.head())
one_match = extract_one_match(data, 4)


In [ ]:
one_match = one_match[one_match['match_name'] == 'Denmark_England'].reset_index(drop=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def plot_trajectory_alignment(trajectory_a, trajectory_b, method="dtw"):
    """
    Plots the alignment of two ball trajectories using either Dynamic Time Warping (DTW) 
    or Euclidean Distance.

    Parameters:
        trajectory_a (np.ndarray): First trajectory as a 2D array of shape (n, 2).
        trajectory_b (np.ndarray): Second trajectory as a 2D array of shape (m, 2).
        method (str): Alignment method - "dtw" for Dynamic Time Warping, "euclidean" for Euclidean Distance.
    """
    if method == "dtw":
        # Compute DTW alignment using Euclidean distance
        distance, path = fastdtw(trajectory_a, trajectory_b, dist=euclidean)
        title = f"Dynamic Time Warping Alignment "
    elif method == "euclidean":
        # Ensure both trajectories have the same length by truncating the longer one
        min_len = min(len(trajectory_a), len(trajectory_b))
        trajectory_a = trajectory_a[:min_len]
        trajectory_b = trajectory_b[:min_len]
        path = [(i, i) for i in range(min_len)]  # Align indices one-to-one
        distance = sum(euclidean(trajectory_a[i], trajectory_b[i]) for i in range(min_len))
        title = f"Euclidean Distance Alignment "
    else:
        raise ValueError("Invalid method. Choose 'dtw' or 'euclidean'.")
    
    # Plot trajectories
    plt.figure(figsize=(10, 6))
    x_a, y_a = trajectory_a[:, 0], trajectory_a[:, 1]
    x_b, y_b = trajectory_b[:, 0], trajectory_b[:, 1]
    plt.plot(x_a, y_a, 'bo-', label="Trajectory A (Reference)")
    plt.plot(x_b, y_b, 'ro-', label="Trajectory B (Comparison)")
    
    # Plot alignment lines
    for (i, j) in path:
        plt.plot([x_a[i], x_b[j]], [y_a[i], y_b[j]], 'k--', alpha=0.6)
    
    plt.legend()
    plt.title(title)
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.grid()
    plt.show()

# Extract two ball trajectories for comparison
trajectory_a = one_match[["ball_x", "ball_y"]].values
trajectory_b = one_match[["ball_x", "ball_y"]].values

# Set time interval for comparison
start_time, end_time = 30576, 30596
start_time1, end_time1 = 44544, 44553

trajectory_a = trajectory_a[start_time:end_time]
trajectory_b = trajectory_b[start_time1:end_time1]

trajectory_a = trajectory_a - trajectory_a[0]
trajectory_b = trajectory_b - trajectory_b[0]
print(trajectory_a)
print(trajectory_b)
# Plot using DTW
plot_trajectory_alignment(trajectory_a, trajectory_b, method="dtw")

# Plot using Euclidean Distance
plot_trajectory_alignment(trajectory_a, trajectory_b, method="euclidean")
 

NameError: name 'one_match' is not defined